In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import torch

In [ ]:
train=pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test=pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
train_text=train['excerpt']
train_labels=train['target']
test_text=test['excerpt']

In [ ]:
from sklearn.model_selection import train_test_split
train_text,val_text,train_labels,val_labels=train_test_split(train_text,train_labels,test_size=0.10)

In [ ]:
from transformers import LongformerTokenizerFast
Tokenier=LongformerTokenizerFast.from_pretrained('../input/allenailongformerbase4096/longformer')
from transformers import LongformerModel
bert=LongformerModel.from_pretrained('../input/allenailongformerbase4096/longformer')

In [ ]:
train_tokens=Tokenier.batch_encode_plus(train_text.to_list(),max_length=200,truncation=True,padding=True,return_tensors='pt',return_attention_mask=True,return_token_type_ids=True)
val_tokens=Tokenier.batch_encode_plus(val_text.to_list(),max_length=200,truncation=True,padding=True,return_tensors='pt',return_attention_mask=True,return_token_type_ids=True)
test_tokens=Tokenier.batch_encode_plus(test_text.to_list(),max_length=200,truncation=True,padding=True,return_tensors='pt',return_attention_mask=True,return_token_type_ids=True)

train_labels=torch.tensor(train_labels.to_list())
val_labels=torch.tensor(val_labels.to_list())

In [ ]:
from torch.utils.data import RandomSampler,SequentialSampler,DataLoader,TensorDataset

train_dataset=TensorDataset(train_tokens['input_ids'],train_tokens['attention_mask'],train_tokens['token_type_ids'],train_labels)
val_dataset=TensorDataset(val_tokens['input_ids'],val_tokens['attention_mask'],val_tokens['token_type_ids'],val_labels)
test_dataset=TensorDataset(test_tokens['input_ids'],test_tokens['attention_mask'],test_tokens['token_type_ids'])

train_sampler=RandomSampler(train_dataset)
val_sampler=RandomSampler(val_dataset)
test_sampler=SequentialSampler(test_dataset)

train_loader=DataLoader(dataset=train_dataset,sampler=train_sampler,batch_size=8)
val_loader=DataLoader(dataset=val_dataset,sampler=val_sampler,batch_size=8)
test_loader=DataLoader(dataset=test_dataset,sampler=test_sampler,batch_size=8)

In [ ]:
class CustomBert(torch.nn.Module):
    def __init__(self,bert):
        super(CustomBert,self).__init__()
        self.bert=bert
        self.dropout=torch.nn.Dropout(0.2)
        self.linear1=torch.nn.Linear(768 ,512)
        self.linear2=torch.nn.Linear(512,1)
        self.Relu=torch.nn.ReLU()
    def forward(self,tokens,mask,token_type_ids):
        _,pool=self.bert(tokens,attention_mask=mask,token_type_ids=token_type_ids,return_dict=False)
        x=self.linear1(pool)
        x=self.Relu(x)
        x=self.dropout(x)
        x=self.linear2(x)
        return x

In [ ]:
CustomBert=CustomBert(bert=bert)
device=torch.device('cuda')
CustomBert=CustomBert.to(device)

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def traindaModel(train_loader,model,optimizer):
    total_loss,total_accuracy=0,0
    total_preds=[]
    model.train()
    for step,batch in enumerate(train_loader):
        if step %50==0 and not step == 0:
            print('Batch {:>5,} of {:>5,}.'.format(step,len(train_loader)))
        batch=[r.to(device) for r in batch]
        tokens,mask,token_type_ids,labels=batch
        model.zero_grad()
        pred=model(tokens,mask,token_type_ids)
        criterion = torch.nn.MSELoss()
        pred=pred.view(-1)
        loss = torch.sqrt(criterion(pred, labels))
        total_loss=total_loss+loss.item()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        optimizer.step()
        preds=pred.detach().cpu().numpy()
        total_preds.append(preds)
    avg_loss=total_loss/len(train_loader)
    total_preds=np.concatenate(total_preds,axis=0)
    return avg_loss,total_preds,model
def evaluatedaModel(val_loader,scheduler,model):
    model.eval()
    total_loss,total_accuracy=0,0
    total_preds=[]
    for step,batch in enumerate(val_loader):
        if step %50==0 and not step == 0:
            print('Batch {:>5,} of {:>5,}.'.format(step,len(val_loader)))
        batch=[r.to(device) for r in batch]
        tokens,mask,token_type_ids,labels=batch
        with torch.no_grad():
            pred=model(tokens,mask,token_type_ids)
            pred=pred.view(-1)
            criterion = torch.nn.MSELoss()
            loss = torch.sqrt(criterion(pred, labels))
            total_loss=total_loss+loss.item()
            scheduler.step(loss)
            preds=pred.detach().cpu().numpy()
            total_preds.append(preds)
    avg_loss=total_loss/len(val_loader)
    total_preds=np.concatenate(total_preds,axis=0)
    return avg_loss,total_preds,model

In [ ]:
from transformers import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
optimizer=AdamW(CustomBert.parameters(),lr=1e-5)
scheduler=ReduceLROnPlateau(optimizer,'min',patience=3)

In [ ]:
epochs=10
best_val_loss=float('inf')
train_losses=[]
val_losses=[]
for epoch in range(epochs):
    print('\n Epochs {:}/{:}'.format(epoch+1,epochs))
    train_loss,_,CustomBert=traindaModel(train_loader=train_loader,model=CustomBert,optimizer=optimizer)
    valid_loss,_,CustomBert=evaluatedaModel(val_loader=val_loader,scheduler=scheduler,model=CustomBert)
    if valid_loss < best_val_loss:
        best_val_loss=valid_loss
        torch.save(CustomBert.state_dict(),'best_model_weights.pt')
    train_losses.append(train_loss)
    val_losses.append(valid_loss)
    print('\nTrain loss :',train_losses)
    print('\nValid loss :',val_losses)

In [ ]:
path='best_model_weights.pt'
total_preds=[]
CustomBert.load_state_dict(torch.load(path))
for step,batch in enumerate(test_loader):
    batch=[r.to(device) for r in batch]
    tokens,mask,token_type_ids=batch
    with torch.no_grad():
        pred=CustomBert(tokens,mask,token_type_ids)
        preds=pred.detach().cpu().numpy()
        total_preds.append(preds)
total_preds=np.concatenate(total_preds,axis=0) 

In [ ]:
ids = test['id']
df = pd.DataFrame({'id': ids, 'target': np.array(total_preds).flatten()})
df.head()
df.to_csv('submission.csv', index=False)